In [26]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from os.path import join


In [27]:
# Directories:
main_dir = 'C:/Users/selloh/Desktop/Datasets/DREYEVE_DATA'
subsequences_dir = main_dir + '/subsequences.txt'
dreyeve_design_dir = main_dir + "/dr(eye)ve_design.txt"


In [28]:
#read driveeye data

# https://github.com/ndrplz/dreyeve/blob/master/experiments/dataset_stats/stats_utils.py


def read_dreyeve_design(dreyeve_root):
    """
    Reads the whole dr(eye)ve design.

    Returns
    -------
    ndarray
        the dr(eye)ve design in the form (sequences, params).
    """

    with open(join(dreyeve_root, 'dr(eye)ve_design.txt')) as f:
        dreyeve_design = np.array([l.rstrip().split('\t') for l in f.readlines()])

    return dreyeve_design

In [29]:
#count the number of normal frames
def count_subsequences_each_sequence(subsequences_file, sequences):
    """
    Returns
    -------
    dict
        a dictionary like:
            {dreyeve_seq: np.array([count_acting, count_errors, ...])}
    """

    ret = {s: np.zeros(5, dtype=np.int32) for s in sequences}  # normal, acting, errors, inattentive, interesting

    # read subsequences file and populate sequence histogram
    with open(subsequences_file, mode='r') as f:
        for line in f.readlines():
            seq, start, end, kind = line.rstrip().split('\t')
            seq, start, end = int(seq), int(start), int(end)

            if kind == 'k':  # acting
                ret[seq][1] += end - start + 1
            elif kind == 'e':  # error
                ret[seq][2] += end - start + 1
            elif kind == 'i':  # inattentive
                ret[seq][3] += end - start + 1
            elif kind == 'u':  # uninteresting
                ret[seq][4] += end - start + 1
            else:
                raise ValueError

    # count `normal` central frames
    for key, value in ret.items():
        value[0] = 7500 - np.sum(value[1:])

    return ret




In [30]:
def main():
    """ Main function """
    main_dir = 'C:/Users/selloh/Desktop/Datasets/DREYEVE_DATA'
    subsequences_dir = main_dir + '/subsequences.txt'
    dreyeve_design_dir = main_dir + "/dr(eye)ve_design.txt"

    sequences = np.arange(1, 74+1)

    subsequences_count = count_subsequences_each_sequence(subsequences_dir, sequences)

    dreyeve_design = read_dreyeve_design(dreyeve_root=main_dir)
    dreyeve_design = {int(s[0]): s[1:] for s in dreyeve_design}

    # aggregate sequences by weather
    weathers = ['Sunny', 'Cloudy', 'Rainy']
    aggr_col = 1
    weather_hist = {w: np.zeros(5, dtype=np.int32) for w in weathers}

    for s in sequences:
        this_sequence_weather = dreyeve_design[s][aggr_col]

        weather_hist[this_sequence_weather] += subsequences_count[s]

    for w in weathers:
        print('{}: {}'.format(w, weather_hist[w].astype(np.float32) / np.sum(weather_hist[w])))


if __name__ == '__main__':
    main()

Sunny: [0.811856   0.11407467 0.04734933 0.02297067 0.00374933]
Cloudy: [0.79574872 0.12996923 0.04306667 0.02450769 0.00670769]
Rainy: [0.80896232 0.13325217 0.03256232 0.01852754 0.00669565]
